In [1]:
from datasets import load_dataset, load_from_disk
import pandas as pd
from tqdm import tqdm

/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
orig_dataset = load_from_disk("../data/stackmagnitude/")['reward']

In [9]:
# Define your categories
#categories = ['https://math.stackexchange.com', 'https://scifi.stackexchange.com', 'https://spanish.stackexchange.com', 'https://mythology.stackexchange.com', 'https://biology.stackexchange.com']
categories = []
# Create a dictionary to store datasets for each category
#filtered_datasets = {}

for category in categories:
    filtered_datasets[category] = orig_dataset.filter(lambda x: x['metadata'][1] == category, num_proc=60)


In [13]:
categories = list(sources)

# Collect indices for each category
category_indices = {category: [] for category in categories}

for i, row in tqdm(enumerate(orig_dataset), total=len(orig_dataset)):
    category = row['metadata'][1]
    if category in category_indices:
        category_indices[category].append(i)

# Create a dictionary to store datasets for each category
filtered_datasets = {}

for category, indices in category_indices.items():
    filtered_datasets[category] = orig_dataset.select(indices)

  1%|▋                                                                                                                                        | 38045/7441157 [00:04<14:09, 8714.32it/s]


KeyboardInterrupt: 

In [19]:
from datasets import load_dataset
from multiprocessing import Pool, cpu_count

categories = list(sources)
# Load your big dataset
dataset = orig_dataset
def collect_indices(start_end):
    start, end = start_end
    local_indices = {category: [] for category in categories}

    for i in tqdm(range(start, end)):
        row = dataset[i]
        category = row['metadata'][1]
        if category in local_indices:
            local_indices[category].append(i)
    
    return local_indices

# Create chunks for parallel processing
num_cores = cpu_count()
chunk_size = len(dataset) // num_cores
#chunks = [(i, i+chunk_size) for i in range(0, len(dataset), chunk_size)]
chunks = [(i, min(i+chunk_size, len(dataset))) for i in range(0, len(dataset), chunk_size)]

# Process chunks in parallel with tqdm progress bar
with Pool(num_cores) as p:
    results = list(p.imap(collect_indices, chunks))

# Combine results from all chunks
category_indices = {category: [] for category in categories}
for local_indices in results:
    for category in categories:
        category_indices[category].extend(local_indices[category])

# Create datasets for each category
filtered_datasets = {}
for category, indices in category_indices.items():
    filtered_datasets[category] = dataset.select(indices)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 265755/265755 [01:20<00:00, 3310.40it/s]


In [34]:
refilt = {}
totdata = 0
for f in filtered_datasets.keys():
    if len(filtered_datasets[f])>10000 and len(filtered_datasets[f])<100000:
        refilt[f.replace("https://", "").replace(".com", "").replace(".stackexchange", "")] = filtered_datasets[f]
        totdata = totdata+len(filtered_datasets[f])

In [35]:
usedsets = ['https://english.stackexchange.com', 'https://workplace.stackexchange.com', 'https://apple.stackexchange.com', 'https://scifi.stackexchange.com']

In [37]:
for f in refilt: 
    refilt[f].save_to_disk("../data/"+f+"/")

In [36]:
refilt

{'workplace': Dataset({
     features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k', 'magnitude'],
     num_rows: 30733
 }),
 'apple': Dataset({
     features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k', 'magnitude'],
     num_rows: 31493
 }),
 'security': Dataset({
     features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k', 'magnitude'],
     num_rows: 22251
 }),
 'mathoverflow.net': Dataset({
     features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k', 'magnitude'],
     num_rows: 24895
 }),
 'codereview': Dataset({
     features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k', 'magnitude'],
     num_rows: 17728
 }),
 'ux': Dataset({
     features: ['qid', 'question', 'answers', 'date', 'metadata', 'response_j', 'response_k', 'magnitude'],
     num_rows: 19357
 }),
 'meta': Dataset({
     features: ['qid', 'question', 'a

In [43]:
for s in refilt.keys():
    temp = """
    torchrun --nnodes 1  --nproc_per_node 8 --master_port=12335 scripts/train_rm.py \\
        --model_name=/u/prasanns/research/rlhf-length-biases/models/stack/sft \\
        --output_dir=checkpoints/stackrms/stack_"""+s+""" \\
        --dataset=\"stack_"""+s+"""\" \\
        --rand_ratio=0 \\
        --balance_len=0 \\
        --num_train_epochs=1"""
    print(temp)


    torchrun --nnodes 1  --nproc_per_node 8 --master_port=12335 scripts/train_rm.py \
        --model_name=/u/prasanns/research/rlhf-length-biases/models/stack/sft \
        --output_dir=checkpoints/stackrms/stack_workplace \
        --dataset="stack_workplace" \
        --rand_ratio=0 \
        --balance_len=0 \
        --num_train_epochs=1

    torchrun --nnodes 1  --nproc_per_node 8 --master_port=12335 scripts/train_rm.py \
        --model_name=/u/prasanns/research/rlhf-length-biases/models/stack/sft \
        --output_dir=checkpoints/stackrms/stack_apple \
        --dataset="stack_apple" \
        --rand_ratio=0 \
        --balance_len=0 \
        --num_train_epochs=1

    torchrun --nnodes 1  --nproc_per_node 8 --master_port=12335 scripts/train_rm.py \
        --model_name=/u/prasanns/research/rlhf-length-biases/models/stack/sft \
        --output_dir=checkpoints/stackrms/stack_security \
        --dataset="stack_security" \
        --rand_ratio=0 \
        --balance_len=0 \
    

In [44]:
uchat = load_dataset("stingning/ultrachat")











Extracting data files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 129.58it/s]


Dataset json downloaded and prepared to /home/prasann/.cache/huggingface/datasets/stingning___json/stingning--ultrachat-b5ef876d26617596/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 24.14it/s]


In [6]:
sources = set()
for s in tqdm(orig_dataset['metadata'][:1000]):
    sources.add(s[1])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2164243.55it/s]


NameError: name 'source' is not defined

In [7]:
sources

{'https://Stackoverflow.com',
 'https://anime.stackexchange.com',
 'https://apple.stackexchange.com',
 'https://arduino.stackexchange.com',
 'https://askubuntu.com',
 'https://biology.stackexchange.com',
 'https://bricks.stackexchange.com',
 'https://buddhism.stackexchange.com',
 'https://codegolf.stackexchange.com',
 'https://codereview.stackexchange.com',
 'https://diy.stackexchange.com',
 'https://electronics.stackexchange.com',
 'https://ell.stackexchange.com',
 'https://english.stackexchange.com',
 'https://ethereum.stackexchange.com',
 'https://fitness.stackexchange.com',
 'https://gamedev.stackexchange.com',
 'https://gardening.stackexchange.com',
 'https://gis.stackexchange.com',
 'https://hinduism.stackexchange.com',
 'https://homebrew.stackexchange.com',
 'https://law.stackexchange.com',
 'https://magento.stackexchange.com',
 'https://math.stackexchange.com',
 'https://mathematica.stackexchange.com',
 'https://mathoverflow.net',
 'https://meta.mathoverflow.net',
 'https://met

In [6]:
df = pd.DataFrame(orig_dataset[:100])

In [11]:
df['metadata'][3]

['https://math.stackexchange.com/questions/3665283',
 'https://math.stackexchange.com',
 'https://math.stackexchange.com/users/534839/']

In [ ]:
set([orig_dataset['metadata'][i][1] for i in range(len(orig_dataset))])